In [ ]:
import sys
from firecloud import fiss
from google.cloud import storage
import firecloud.api as fapi
import os
import io
import json

# Scientific computing in python
import numpy as np
# Data visualizations
import matplotlib.pyplot as plt
# Data analysis tools and data structures like the DataFrame
import pandas as pd
# Statistical data visualization, site: https://seaborn.pydata.org/
import seaborn as sns

# Get the Google billing project name and workspace name
project = os.environ['WORKSPACE_NAMESPACE']
workspace = os.environ['WORKSPACE_NAME']
bucket = os.environ['WORKSPACE_BUCKET'] + "/"
google_project = os.environ['GOOGLE_PROJECT']

# Verify that we've captured the environment variables
print("Terra Billing project: " + project)
print("Workspace: " + workspace)
print("Workspace storage bucket: " + bucket)
print("Google project: " + google_project)

In [ ]:
from google.cloud import storage

def list_gcs_files(bucket_name, prefix):
    # Initialize a client
    client = storage.Client()

    # Get the bucket
    bucket = client.get_bucket(bucket_name)

    # List blobs in the specified directory
    blobs = bucket.list_blobs(prefix=prefix)

    for blob in blobs:
        print(blob.name)

# Replace with your bucket name and directory prefix
bucket_name = 'fc-xxxxxx-xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx'
prefix = 'working-set-redeposit'

list_gcs_files(bucket_name, prefix)

In [ ]:
import hail as hl
import pandas as pd

# Initialize Hail
hl.init(default_reference="GRCh38")

In [ ]:
# ------------------- Define 163k WGS paths ------------------- #
wgs_paths = {
    1: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_1",
    2: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_2",
    3: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_3",
    4: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_4",
    5: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_5",
    6: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_6",
    7: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_7",
    8: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_8",
    9: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_9",
    10: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_10",
    11: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_11",
    12: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_12",
    13: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_13",
    14: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_14",
    15: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_15",
    16: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_16",
    17: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_17",
    18: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_18",
    19: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_19",
    20: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_20",
    21: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_21",
    22: "gs://fc-secure/call-prune_for_relatedness_testing/agd163k_22"
}

# ------------------- Function to Load & Filter PLINK Data ------------------- #
def load_and_filter_plink(paths_dict):
    mt_list = []
    for chrom, plink_prefix in paths_dict.items():
        print(f"Loading chromosome {chrom} from {plink_prefix}")
        mt = hl.import_plink(
            bed = plink_prefix + ".bed",
            bim = plink_prefix + ".bim",
            fam = plink_prefix + ".fam",
            reference_genome = "GRCh38"
        )
        # Filter rows by joining with snp_keep_table via rsid
        #mt_filtered = mt.filter_rows(hl.is_defined(snp_keep_table[mt.rsid]))
        mt_list.append(mt)
    # Iteratively union the rows from each chromosome
    combined = mt_list[0]
    for mt in mt_list[1:]:
        combined = combined.union_rows(mt)
    return combined

print("Loading WGS dataset...")
mt_wgs = load_and_filter_plink(wgs_paths)

In [ ]:
print("Exporting as PLINK...")
hl.export_plink(mt_wgs, "gs://fc-secure/relatedness_testing/subset_163k/163k_pruned_missingness0.999_biallelic_maf0.3_ld_50000_10_0.8")

In [ ]:
#######################
#In case additional filtering is necessary

In [ ]:
import sys
from firecloud import fiss
from google.cloud import storage
import firecloud.api as fapi
import os
import io
import json

# Scientific computing in python
import numpy as np
# Data visualizations
import matplotlib.pyplot as plt
# Data analysis tools and data structures like the DataFrame
import pandas as pd
# Statistical data visualization, site: https://seaborn.pydata.org/
import seaborn as sns

# Get the Google billing project name and workspace name
project = os.environ['WORKSPACE_NAMESPACE']
workspace = os.environ['WORKSPACE_NAME']
bucket = os.environ['WORKSPACE_BUCKET'] + "/"
google_project = os.environ['GOOGLE_PROJECT']

# Verify that we've captured the environment variables
print("Terra Billing project: " + project)
print("Workspace: " + workspace)
print("Workspace storage bucket: " + bucket)
print("Google project: " + google_project)

In [ ]:
from google.cloud import storage

def list_gcs_files(bucket_name, prefix):
    # Initialize a client
    client = storage.Client()

    # Get the bucket
    bucket = client.get_bucket(bucket_name)

    # List blobs in the specified directory
    blobs = bucket.list_blobs(prefix=prefix)

    for blob in blobs:
        print(blob.name)

# Replace with your bucket name and directory prefix
bucket_name = 'fc-secure'
prefix = 'relatedness_testing/subset_163k'

list_gcs_files(bucket_name, prefix)

In [ ]:
def create_folder(bucket_name, destination_folder_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_folder_name)

    blob.upload_from_string('')

    print('Created {} .'.format(
        destination_folder_name))


In [ ]:
import hail as hl
hl.init(default_reference='GRCh38', idempotent=True)

In [ ]:
mt = hl.import_plink(
bed='gs://fc-secure/relatedness_testing/subset_163k/163k_pruned_missingness0.999_biallelic_maf0.3_ld_50000_10_0.8.bed',
bim='gs://fc-secure/relatedness_testing/subset_163k/163k_pruned_missingness0.999_biallelic_maf0.3_ld_50000_10_0.8.bim',
fam='gs://fc-secure/relatedness_testing/subset_163k/163k_pruned_missingness0.999_biallelic_maf0.3_ld_50000_10_0.8.fam',
reference_genome='GRCh38')

In [ ]:
# Compute variant QC metrics
mt = hl.variant_qc(mt)

# Filter SNPs: call rate ≥ 0.9999999
mt = mt.filter_rows((mt.variant_qc.call_rate >= 0.999999999))

In [ ]:
print(f"Number of variants after pruning: {mt.count_rows()}")

In [ ]:
print("Exporting as PLINK...")
hl.export_plink(mt, "gs://fc-secure/relatedness_testing/subset_163k/163k_pruned_missingness0.999_biallelic_maf0.3_ld_50000_10_0.8_50k_snps")


In [ ]:
# # Filter to include only biallelic variants
# biallelic_mt = mt.filter_rows(hl.len(mt.alleles) == 2)

# # Now perform LD pruning on the biallelic dataset
# pruned_variant_table = hl.ld_prune(biallelic_mt.GT, r2=0.8, bp_window_size=20000)